In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/src/')

# import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

# import logging
# logging.basicConfig(filename='./__upd_runGASiteProd2SAS__.log',level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
# from etl import ETLORA


# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'sbx_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
tbl = 'GA_SITE_ALL_PRODS'
out_table_name = 'GA_ALL_SCENARIOS_HIST'
##-----------------------------------------http://pklis-chd000224.labiac.df.sbrf.ru:8100/notebooks/ISKRA_GET_LOAD/Hive2Iskra.ipynb#

def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

# @typed_udf(StringType())
def match_product(product_dict_nm):
    def get_product(product_norm, product_dict_nm):
        if product_norm is None:
            return None

        for product_nm in product_dict_nm:
            if product_nm in product_norm:
                return product_dict_nm[product_nm]
            else:
                return None

    return f.udf(lambda x: get_product(x,product_dict_nm))

In [6]:
sp = spark(schema=conn_schema,
               dynamic_alloc=False,
               numofinstances=8,
               numofcores=8,
               executor_memory='25g',
               driver_memory='20g',
               kerberos_auth=False,
               process_label="SAS_OFFER_REPARTITION_"
               )

hive = sp.sql
print(sp.sc.version)

====================================================== 2022-05-16 ======================================================
# __init__ : begin
2.4.0.cloudera2


In [126]:
sdf = hive.table("sbx_t_team_cvm.bigartm_inf_v_1_0")

In [128]:
sdf = sdf.select([col.upper() for col in sdf.columns])
sdf = sdf.withColumnRenamed("COMMENT", "COMMENT_KM_TXT")

In [48]:
NAT_MASK      = "2099-01-01"
NAT_YEAR_MASK = "2099"
SEP=";"

In [49]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
print("Return some str columns type to Numeric type due to possible wrong type infering after Oracle import")

for col, _type in sdf.dtypes:
    if _type == 'string':
        if col.lower() != 'inn':
            df = sdf.select(f.col(col)).filter(~f.col(col).isNull()).limit(10).toPandas()
            if df.shape[0] > 0:
                val = df.loc[df[col].first_valid_index(), col]
                try:
                    res = int(val)
                    sdf = sdf.withColumn(col, f.col(col).cast(IntegerType()))
                except ValueError as ex:
                    # 'invalid literal for int() with base 10'
                    print(str(ex))
                    try:
                        res = float(val)
                        sdf = sdf.withColumn(col, f.col(col).cast(FloatType()))
                    except ValueError as ex:
                        # 'could not convert string to float'
                        print(str(ex))

        sdf = sdf.fillna({col: ''})
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
print("## Work with TimeStamp cols to find `ValueError: year is out of range`")
col_dt = []
for col, _type in sdf.dtypes:
    if _type == 'timestamp':
        col_dt.append(col)

for col in col_dt:
    sdf = sdf.withColumn(col,
                         (f.regexp_replace( f.col(col).cast(StringType()),
                                            "^([013456789]{4})(?:\-).*",
                                            NAT_YEAR_MASK)).cast(TimestampType())
                        )
    sdf = sdf.withColumn(col, f.coalesce(col, f.lit(pd.to_datetime(NAT_MASK))))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for col, _type in sdf.dtypes:
    if _type == 'string':
        sdf = sdf.withColumn(col, f.regexp_replace(col,'[\{}]'.format(SEP), ""))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

df = sdf.limit(1000).toPandas()
print(df.dtypes)

Return some str columns type to Numeric type due to possible wrong type infering after Oracle import
invalid literal for int() with base 10: ''
could not convert string to float: 
invalid literal for int() with base 10: ''
could not convert string to float: 
invalid literal for int() with base 10: '[Скидка], Рекламная кампания от 2ГИС со скидкой. Кемеровская Область'
could not convert string to float: '[Скидка], Рекламная кампания от 2ГИС со скидкой. Кемеровская Область'
invalid literal for int() with base 10: '1-5N3D0I3S'
could not convert string to float: '1-5N3D0I3S'
invalid literal for int() with base 10: ''
could not convert string to float: 
## Work with TimeStamp cols to find `ValueError: year is out of range`
MODEL_ID              object
COMMENT_SOURCE        object
INN                   object
COMMENT               object
PROCESSED_DAY_D       object
PROD_OFFER_CRM_ID     object
TASK_CRM_ID           object
CLUST_0              float64
CLUST_1              float64
CLUST_2     

In [113]:
df = df.fillna(0.0)

In [116]:
df['PROCESSED_DAY_D'] = df['PROCESSED_DAY_D'].apply(lambda x: pd.to_datetime(datetime.strftime(x, format='%Y-%m-%d'), 
                                                                     format='%Y-%m-%d')
                                           )

# df.rename(columns={"COMMENT":"COMMENT_KM_TXT"}, inplace=True)

## Create MetaTable in Oracle

In [117]:
str_ = 'CREATE TABLE {} ( '
for column_name, column in df.iteritems():
    print(column_name)
    if (isinstance(column[column.first_valid_index()], str)):
        if (df[column_name].str.len().max() >= 4000):
            str_+=column_name.upper() + ' ' +'CLOB, '
        else:
            if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
                str_+=column_name.upper() + ' ' +'VARCHAR2(20), '
            else:
                str_+=column_name.upper() + ' ' +'VARCHAR2(800), '
    elif column.dtype.kind == 'i':
        str_+=column_name.upper() + ' ' +'NUMBER(20), '
    elif column.dtype.kind == 'f':
        if 'TARGET' in column_name.upper():
            str_+=column_name.upper() + ' ' +'NUMBER(1), '
        else:
            df[column_name] = df[column_name].fillna(0.0)
            str_+=column_name.upper() + ' ' +'FLOAT(126), '        
    elif column.dtype.kind == 'M':
        if 'PROCESSED_DAY_D' in column_name.upper():
            str_+=column_name.upper() + ' ' +'DATE, '
        else:
            str_+=column_name.upper() + ' ' +'TIMESTAMP, '
    elif column.dtype.kind == 'b':
        str_+=column_name.upper() + ' ' +'BOOLEAN, '        
    else:
        None  
res=str_.strip()[:-1] + ' )' \
''' 
PARTITION BY RANGE (PROCESSED_DAY_D) 
INTERVAL(NUMTODSINTERVAL(1,'DAY')) 
(PARTITION SYS_P1 VALUES LESS THAN (TO_DATE('2021-01-01', 'YYYY-MM-DD')),
 PARTITION SYS_P2 VALUES LESS THAN (TO_DATE('2021-01-02', 'YYYY-MM-DD'))
)
'''

print(res)

MODEL_ID
COMMENT_SOURCE
INN
COMMENT_KM_TXT
PROCESSED_DAY_D
PROD_OFFER_CRM_ID
TASK_CRM_ID
CLUST_0
CLUST_1
CLUST_2
CLUST_3
CLUST_4
CLUST_5
CLUST_6
CLUST_7
CLUST_8
CLUST_9
CLUST_10
CLUST_11
CLUST_12
CLUST_13
CLUST_14
CLUST_15
CLUST_16
CLUST_17
CLUST_18
CLUST_19
CLUST_20
CLUST_21
CLUST_22
CLUST_23
CLUST_24
CLUST_25
CLUST_26
CLUST_27
CLUST_28
CLUST_29
CLUST_30
CLUST_31
CLUST_32
CLUST_33
CLUST_34
CLUST_35
CLUST_36
CLUST_37
CLUST_38
CLUST_39
CLUST_40
CLUST_41
CLUST_42
CLUST_43
CLUST_44
CLUST_45
CLUST_46
CLUST_47
CLUST_48
CLUST_49
CREATE TABLE {} ( MODEL_ID VARCHAR2(800), COMMENT_SOURCE VARCHAR2(800), INN VARCHAR2(20), COMMENT_KM_TXT VARCHAR2(800), PROCESSED_DAY_D DATE, PROD_OFFER_CRM_ID VARCHAR2(800), TASK_CRM_ID VARCHAR2(800), CLUST_0 FLOAT(126), CLUST_1 FLOAT(126), CLUST_2 FLOAT(126), CLUST_3 FLOAT(126), CLUST_4 FLOAT(126), CLUST_5 FLOAT(126), CLUST_6 FLOAT(126), CLUST_7 FLOAT(126), CLUST_8 FLOAT(126), CLUST_9 FLOAT(126), CLUST_10 FLOAT(126), CLUST_11 FLOAT(126), CLUST_12 FLOAT(126), CLUST_

In [118]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('ISKRA_CVM', 'hud2X_3VRCe5Lw_rpcN0oCu5', ld._get_dsn('iskra4')) 
db.connect()
curs = db.cursor

In [119]:
table_name_new = 'bigartm_topicks_km'

# sql = "DROP TABLE {}".format(table_name_new)
# db.cursor.execute(sql)
# db.connection.commit()
# print('dropping done')

sql = res.format(table_name_new)

db.cursor.execute(sql)
db.connection.commit()
print('creating done')

creating done


## Export via jDBC driver

In [129]:
typesmap={}
for column_name, column in sdf.dtypes:
    if column == 'string':
        if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
            typesmap[column_name] = 'VARCHAR(20)'
        elif 'commonSegmentoUID'.upper() in column_name.upper():
            typesmap[column_name] = 'CLOB'
        else:
            typesmap[column_name] = 'VARCHAR(900)'
    elif column == 'int':
        typesmap[column_name] = 'INTEGER'
    elif column == 'bigint':
        typesmap[column_name] = 'LONG'
    elif column == 'timestamp':
        typesmap[column_name] = 'TIMESTAMP'
    elif column == 'date':
        typesmap[column_name] = 'DATE'        
    elif column == 'float' or column == 'double':
        typesmap[column_name] = 'FLOAT'
    else:
        None
        
cols = ', '.join([col + ' ' + typesmap[col] for col in sdf.columns])
cols

'MODEL_ID VARCHAR(900), COMMENT_SOURCE VARCHAR(900), INN VARCHAR(20), COMMENT_KM_TXT VARCHAR(900), PROCESSED_DAY_D DATE, PROD_OFFER_CRM_ID VARCHAR(900), TASK_CRM_ID VARCHAR(900), CLUST_0 FLOAT, CLUST_1 FLOAT, CLUST_2 FLOAT, CLUST_3 FLOAT, CLUST_4 FLOAT, CLUST_5 FLOAT, CLUST_6 FLOAT, CLUST_7 FLOAT, CLUST_8 FLOAT, CLUST_9 FLOAT, CLUST_10 FLOAT, CLUST_11 FLOAT, CLUST_12 FLOAT, CLUST_13 FLOAT, CLUST_14 FLOAT, CLUST_15 FLOAT, CLUST_16 FLOAT, CLUST_17 FLOAT, CLUST_18 FLOAT, CLUST_19 FLOAT, CLUST_20 FLOAT, CLUST_21 FLOAT, CLUST_22 FLOAT, CLUST_23 FLOAT, CLUST_24 FLOAT, CLUST_25 FLOAT, CLUST_26 FLOAT, CLUST_27 FLOAT, CLUST_28 FLOAT, CLUST_29 FLOAT, CLUST_30 FLOAT, CLUST_31 FLOAT, CLUST_32 FLOAT, CLUST_33 FLOAT, CLUST_34 FLOAT, CLUST_35 FLOAT, CLUST_36 FLOAT, CLUST_37 FLOAT, CLUST_38 FLOAT, CLUST_39 FLOAT, CLUST_40 FLOAT, CLUST_41 FLOAT, CLUST_42 FLOAT, CLUST_43 FLOAT, CLUST_44 FLOAT, CLUST_45 FLOAT, CLUST_46 FLOAT, CLUST_47 FLOAT, CLUST_48 FLOAT, CLUST_49 FLOAT'

In [130]:
db = OracleDB('iskra4')
mode = 'append'
sdf \
    .write \
    .format('jdbc') \
    .mode(mode) \
    .option('url', 'jdbc:oracle:thin:@//'+db.dsn) \
    .option('user', db.user) \
    .option('password', db.password) \
    .option('dbtable', table_name_new) \
    .option('createTableColumnTypes', cols)\
    .option('driver', 'oracle.jdbc.driver.OracleDriver') \
    .save()

## Check saved DF

In [132]:
query = '''
(select /*+ parallel (4) */
    *      
FROM {}
where rownum < 1000)
'''.format(table_name_new)

sdf = sp.get_oracle(OracleDB('iskra4'), query)

In [134]:
sdf.show()

,MODEL_ID,COMMENT_SOURCE,INN,COMMENT_KM_TXT,PROCESSED_DAY_D,PROD_OFFER_CRM_ID,TASK_CRM_ID,CLUST_0,CLUST_1,CLUST_2,...,CLUST_40,CLUST_41,CLUST_42,CLUST_43,CLUST_44,CLUST_45,CLUST_46,CLUST_47,CLUST_48,CLUST_49
0,None,None,421715274206,//Внимание ЦКР! Связаться 25.04. Кемеровская О...,2022-04-25,1-5OETQX4U,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
1,None,None,234912497409,acquiring,2022-04-25,1-5NZ4PDD6,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
2,None,None,234912497409,acquiring,2022-04-25,1-5NZ4PDD4,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
3,None,None,183270458753,Недозвон,2022-04-25,1-5OCCT6AP,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
4,None,None,610301196381,Ростов-на-Дону. Ростовская Область/Ростов-на-Д...,2022-04-25,1-5O50OERC,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
5,None,None,1684003760,"Казань. Татарстан Республика/Казань/г.Казань, ...",2022-04-25,1-5O1VZLV9,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
6,None,None,212907515864,acquiring. актуально после получения документо...,2022-04-25,1-5O5CE6YW,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
7,None,None,212907515864,acquiring. актуально после получения документо...,2022-04-25,1-5O5CE6YY,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
8,None,None,382704706035,BEZVIZITAVBANK. Иркутская Область,2022-04-25,1-5O8TLABC,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
9,None,None,382704706035,BEZVIZITAVBANK. Иркутская Область,2022-04-25,1-5O8TLAB8,None,0E-10,0E-10,0E-10,...,None,None,None,None,None,None,None,None,None,None
